<a href="https://colab.research.google.com/github/sali-ns/Algo_final/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import defaultdict
import itertools
from google.colab import files
files.upload()
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Step 3: Install Kaggle CLI if not already installed
!pip install kaggle
!kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024

!unzip 1-3m-linkedin-jobs-and-skills-2024.zip

# Load job_skills.csv file into a DataFrame
df = pd.read_csv('job_skills.csv')
df['job_skills'] = df['job_skills'].fillna('').astype(str)
df['job_skills'] = df['job_skills'].apply(lambda x: x.split(','))


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024
License(s): ODC Attribution License (ODC-By)
100% 1.87G/1.88G [00:20<00:00, 101MB/s] 
100% 1.88G/1.88G [00:20<00:00, 99.8MB/s]
Archive:  1-3m-linkedin-jobs-and-skills-2024.zip
  inflating: job_skills.csv          
  inflating: job_summary.csv         
  inflating: linkedin_job_postings.csv  


In [2]:

def get_frequent_itemsets(transactions, min_support):
    itemsets = defaultdict(int)
    for transaction in transactions:
        for item in set(transaction):  # Convert to set for uniqueness
            itemsets[frozenset([item])] += 1

    num_transactions = len(transactions)
    itemsets = {itemset: support / num_transactions for itemset, support in itemsets.items() if support / num_transactions >= min_support}

    def join_itemsets(itemsets, k):
        joined_itemsets = defaultdict(int)
        for itemset1, support1 in itemsets.items():
            for itemset2, support2 in itemsets.items():
                if len(itemset1.union(itemset2)) == k:
                    joined_itemsets[itemset1.union(itemset2)] += 1
        return {itemset: support / num_transactions for itemset, support in joined_itemsets.items() if support / num_transactions >= min_support}

    k = 2
    while True:
        new_itemsets = join_itemsets(itemsets, k)
        if not new_itemsets:
            break
        itemsets.update(new_itemsets)
        k += 1

    return itemsets



In [3]:
def generate_rules(frequent_itemsets, min_confidence):
    rules = []
    for itemset, support in frequent_itemsets.items():
        if len(itemset) > 1:
            for consequence in itemset:
                antecedent = itemset - frozenset([consequence])
                antecedent_support = frequent_itemsets[antecedent]
                confidence = support / antecedent_support
                if confidence >= min_confidence:
                    rules.append((antecedent, frozenset([consequence]), confidence))
    return rules


In [4]:

# Extract transactions from DataFrame
transactions = df['job_skills'].tolist()

# Set minimum support and confidence thresholds
min_support = 0.1
min_confidence = 0.5

# Find frequent itemsets
frequent_itemsets = get_frequent_itemsets(transactions, min_support)

# Generate association rules
rules = generate_rules(frequent_itemsets, min_confidence)

In [5]:


# Display the results
print("Frequent Itemsets:")
for itemset, support in frequent_itemsets.items():
    print(f"Itemset: {set(itemset)}, support: {support:.2f}")

print("\nAssociation Rules:")
for antecedent, consequence, confidence in rules:
    print(f"Rule: {set(antecedent)} -> {set(consequence)}, confidence: {confidence:.2f}")


Frequent Itemsets:
Itemset: {' Teamwork'}, support: 0.17
Itemset: {' Communication'}, support: 0.27
Itemset: {' Leadership'}, support: 0.12

Association Rules:
